# CNN in keras with pretrained word2vec weights
https://www.kaggle.com/marijakekic/cnn-in-keras-with-pretrained-word2vec-weights/notebook

In [1]:
import os
import sys
import json
import numpy as np
import pandas as pd

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
# Helper function
def writeProgress(msg, count, total):
    sys.stdout.write(msg + "{:.2%}\r".format(count/total))
    sys.stdout.flush()
    
def newPath(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def read_json(src_path):
    with open(src_path, 'r') as json_file:
        data = json.load(json_file)
    return data

def write_json(data,dst_path):
    with open(dst_path, 'w') as outfile:
        json.dump(data, outfile)

In [4]:
columns = read_json('../orderdListGenres.json')
print(len(columns), columns)

20 ['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western']


# load data

In [5]:
data = read_json('./input/mergeGenresMat.json')
print(len(data))

1562


In [6]:
train_data_IG=pd.read_csv('./stopword/train_IG.csv')
train_data_IG = train_data_IG.dropna()
train_data_IG = train_data_IG.reset_index(drop=True)
# train_data_IG

In [7]:
train_data_imdb=pd.read_csv('./stopword/train_imdb.csv')
train_data_imdb = train_data_imdb.dropna()
train_data_imdb = train_data_imdb.reset_index(drop=True)
train_data_imdb = train_data_imdb.rename(columns={'tconst':'username'})
# train_data_imdb

In [8]:
train_data = pd.concat([train_data_IG, train_data_imdb],ignore_index=True)
train_data

,username,convert_text,genres
0,47metersdown,catch meters uncaged spot last night nbafinals...,"['Adventure', 'Drama', 'Horror']"
1,47metersdown,know swims beneath deep meters uncaged comes t...,"['Adventure', 'Drama', 'Horror']"
2,47metersdown,stay alive meters uncaged theaters nationwide ...,"['Adventure', 'Drama', 'Horror']"
3,47metersdown,take bite summer takes bite meters uncaged dev...,"['Adventure', 'Drama', 'Horror']"
4,47metersdown,upside irl sharkbait,"['Adventure', 'Drama', 'Horror']"
...,...,...,...
11735,tt5189828,late war young mohawk woman two lovers battle ...,"Action,Drama,History"
11736,tt6334884,legendary lawman gunslinger wild bill hickok t...,Western
11737,tt6243274,tough nonsense marine vet alden rockwell lost ...,"Action,Drama,Western"
11738,tt6217608,suffering near fatal head injury young cowboy ...,"Drama,Western"


In [9]:
test_data=pd.read_csv('./stopword/test.csv')
test_data

,username,convert_text
0,47metersdown,four teen girls diving ruined underwater city ...
1,adogsjourneymovie,dog finds meaning existence lives humans meets
2,aftermathmovie,post world war ii british colonel wife assigne...
3,aftermovie,young woman falls guy dark secret two embark r...
4,alitamovie,deactivated cyborg revived remember anything p...
...,...,...
92,usmovie,family serene beach vacation turns chaos doppe...
93,whatmenwant,woman boxed male sports agents profession gain...
94,wonderparkmovie,wonder park tells story amusement park imagina...
95,xmenmovies,jean grey begins develop incredible powers cor...


In [10]:
print(train_data.shape,test_data.shape)
print(train_data.isnull().sum())
print(test_data.isnull().sum())

(11740, 3) (97, 2)
username        0
convert_text    0
genres          0
dtype: int64
username        0
convert_text    0
dtype: int64


In [11]:
ids = train_data['username'].tolist()
texts = train_data['convert_text'].tolist()
print(len(ids), len(texts))

11740 11740


In [12]:
labels = [data[id] for id in ids]
len(labels)

11740

In [13]:
val_data=train_data.sample(frac=0.2,random_state=42)
train_data= train_data.drop(val_data.index)
print(train_data.shape, val_data.shape)

(9392, 3) (2348, 3)


# Tokenize

In [14]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.
/home/tonylab/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tonylab/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tonylab/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/tonylab/.local/lib/python3.6/site-packages/tensorflow/py

In [15]:
texts = train_data.convert_text

In [16]:
NUM_WORDS=30000
tokenizer = Tokenizer(num_words=NUM_WORDS,filters='—!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',
                      lower=True)

tokenizer.fit_on_texts(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 18046 unique tokens.


In [17]:
# sorted(word_index.items(), key=lambda d: d[1])

In [18]:
sequences_train = tokenizer.texts_to_sequences(texts)
sequences_valid = tokenizer.texts_to_sequences(val_data.convert_text)

X_train = pad_sequences(sequences_train)
X_val = pad_sequences(sequences_valid,maxlen=X_train.shape[1])
labels = np.asarray(labels)

y_train = np.asarray(labels[train_data.index.values])
y_val = np.asarray(labels[val_data.index.values])

print('Shape of X train and X validation tensor:', X_train.shape,X_val.shape)
print('Shape of label train and validation tensor:', y_train.shape,y_val.shape)

Shape of X train and X validation tensor: (9392, 330) (2348, 330)
Shape of label train and validation tensor: (9392, 20) (2348, 20)


# Embedding

In [ ]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.keyedvectors import KeyedVectors

outOfDict = []

word_vectors = KeyedVectors.load_word2vec_format('./model/GoogleNews-vectors-negative300.bin', binary=True)

EMBEDDING_DIM=300
vocabulary_size=min(len(word_index)+1,NUM_WORDS)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)
        outOfDict.append(word)

In [ ]:
write_json(outOfDict, 'unstemmed_outOfDict.json')

In [ ]:
del(word_vectors)

from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

## 儲存路徑

In [ ]:
SAVE_NAME = 'merge_stride6'
HISTORY_SAVE = './history/' + SAVE_NAME + '.csv'
WEIGHTS_SAVE = './weight/' + SAVE_NAME + '.h5'

## Model Structure

In [ ]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers

sequence_length = X_train.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.5
stride = 6

inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
reshape = Reshape((sequence_length,EMBEDDING_DIM,1))(embedding)

conv_0 = Conv2D(num_filters, (filter_sizes[0], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv2D(num_filters, (filter_sizes[1], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv2D(num_filters, (filter_sizes[2], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

maxpool_0 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(stride,1))(conv_0)
maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[1] + 1, 1), strides=(stride,1))(conv_1)
maxpool_2 = MaxPooling2D((sequence_length - filter_sizes[2] + 1, 1), strides=(stride,1))(conv_2)

merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2], axis=1)
flatten = Flatten()(merged_tensor)
reshape = Reshape((3*num_filters,))(flatten)
dropout = Dropout(drop)(flatten)
output = Dense(units=20, activation='sigmoid',kernel_regularizer=regularizers.l2(0.01))(dropout)

# this creates a model that includes
model = Model(inputs, output)

In [ ]:
model.summary()

In [ ]:
adam = Adam(lr=1e-3)

model.compile(loss='binary_crossentropy',
              optimizer=adam,
              metrics=['acc'])
callbacks = [EarlyStopping(monitor='val_loss')]
history = model.fit(X_train, y_train, batch_size=32, epochs=100, verbose=1, validation_data=(X_val, y_val), callbacks=callbacks)  # starts training

In [ ]:
# save model weights
model.save(WEIGHTS_SAVE)

In [ ]:
# save history
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

his_df = pd.DataFrame(data = {
    'Epoch' : epochs,
    'Loss' : loss,
    'Acc' : acc,
    'Val_loss' : val_loss,
    'Val_acc' : val_acc
})
his_df = his_df[['Epoch', 'Loss', 'Acc', 'Val_loss', 'Val_acc']]
his_df.to_csv(HISTORY_SAVE, index = False)

In [ ]:
# 繪製結果
import matplotlib.pyplot as plt

# plt.figure()

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.show()

# plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## Testing

In [ ]:
sequences_test=tokenizer.texts_to_sequences(test_data.convert_text)
X_test = pad_sequences(sequences_test,maxlen=X_train.shape[1])
pred = model.predict(X_test)

In [ ]:
pred.shape

In [ ]:
predictions = pred #pred_bool.astype(int)

results=pd.DataFrame(predictions, columns=columns)
results["username"] = test_data.username
ordered_cols = ["username"] + columns
results = results[ordered_cols] #To get the same column order
results.to_csv("./output/"+ SAVE_NAME + "_result.csv",index=False)

## Label top n dynamic

In [ ]:
true_df = pd.read_csv('./input/true_df.csv')
true_df

In [ ]:
results = pd.read_csv("./output/"+ SAVE_NAME + "_result.csv")
results

In [ ]:
labels = read_json('../genresDic.json')

In [ ]:
def countAcc(predList, trueList):
    fraction = 0
    denominator = len(trueList)
    for g in predList:
        if g in trueList:
            fraction += 1 
    return fraction / denominator

In [ ]:
li = []
for i, row in true_df.iterrows():
    username = row['username']
    if username == results.loc[i,:]['username']: # username equals
        print(i, username)
        truth = labels[username]
        numTrue = sum(row[1:])
        print('count:', numTrue)
        
        s = results.loc[i,:][1:]
        top_n = s.sort_values(ascending=False)[:numTrue]
        inList = list(top_n.index)
        
        onehot = []
        for g in columns:
            onehot.append(int(g in inList))
            
        acc = countAcc(inList, truth)
        print(acc, inList, truth)
        
        appList = [username, acc, truth, inList] + onehot
        li.append(appList)
        

In [ ]:
acc_df = pd.DataFrame(li, columns = ['username', 'acc', 'truth', 'top_n'] + columns)
acc_df

In [ ]:
acc_df.to_csv("./output/"+ SAVE_NAME + "_bi.csv", index = 0)

In [ ]:
acc_df['acc'].mean()

# Feature Extraction
## 測試reload model準確度

In [ ]:
SAVE_NAME = 'test'

In [ ]:
WEIGHTS_LOAD = './weight/merge_stride4.h5'

from keras.models import load_model

model = load_model(WEIGHTS_LOAD)

In [ ]:
model.summary()

In [ ]:
sequences_test=tokenizer.texts_to_sequences(test_data.convert_text)
X_test = pad_sequences(sequences_test,maxlen=X_train.shape[1])
pred = model.predict(X_test)
pred.shape

In [ ]:
predictions = pred

results=pd.DataFrame(predictions, columns=columns)
results["username"] = test_data.username
ordered_cols = ["username"] + columns
results = results[ordered_cols] #To get the same column order
results.to_csv("./output/"+ SAVE_NAME + "_result.csv",index=False)

In [ ]:
labels = read_json('../genresDic.json')

In [ ]:
li = []
for i, row in true_df.iterrows():
    username = row['username']
    if username == results.loc[i,:]['username']: # username equals
        print(i, username)
        truth = labels[username]
        numTrue = sum(row[1:])
        print('count:', numTrue)
        
        s = results.loc[i,:][1:]
        top_n = s.sort_values(ascending=False)[:numTrue]
        inList = list(top_n.index)
        
        onehot = []
        for g in columns:
            onehot.append(int(g in inList))
            
        acc = countAcc(inList, truth)
        print(acc, inList, truth)
        
        appList = [username, acc, truth, inList] + onehot
        li.append(appList)

In [ ]:
acc_df = pd.DataFrame(li, columns = ['username', 'acc', 'truth', 'top_n'] + columns)
acc_df['acc'].mean()

## 萃取97部電影imdb的feature

In [ ]:
X_test.shape

In [ ]:
extract_layer = Model(inputs=model.input,
                      outputs=model.get_layer('flatten_4').output)
#以这个model的预测值作为输出
extract_output = extract_layer.predict(X_test)
print(extract_output.shape)

In [ ]:
idx = test_data.username.tolist()
print(len(idx), idx)

In [ ]:
res=pd.DataFrame(extract_output, index = idx)
res

In [ ]:
res.to_csv("./output/TextFeatureVec.csv", header=False)